직원 퇴사예측 예제

In [36]:
#컬럼설명: satisfaction_level(직원만족도점수), last_evaluation(고용주평가점수), numbers_projects(할당된 프로젝트 수)
#average_monthly_hours(한달동안 직원이 일한 평균 시간),
#time_spent_company(회사에서 근무한 연수), work_accident(근무중 사고유무)
#promotion_last_5years(지난 5년 직원이 승진했는지 여부), Departments(부서)
#Salary(월급수준 낮음, 중간, 높음), left(직원퇴사 여부)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt

# 폰트지정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 마이너스 부호 깨짐 지정
plt.rcParams['axes.unicode_minus'] = False

# 숫자가 지수표현식으로 나올 때 지정
pd.options.display.float_format = '{:.2f}'.format

# 데이터 로드
df = pd.read_csv('dataset/HR_comma_sep.csv', encoding='cp949')

#데이터 확인
print(df.head());

# 'left' 열이 정수형인지 확인하고 필요시 변환
if df['left'].dtype != 'int64':
    df['left'] = df['left'].astype(int)
    

   satisfaction_level  last_evaluation  ...  Departments   salary
0                0.38             0.53  ...         sales     low
1                0.80             0.86  ...         sales  medium
2                0.11             0.88  ...         sales  medium
3                0.72             0.87  ...         sales     low
4                0.37             0.52  ...         sales     low

[5 rows x 10 columns]


# 범주형 변수 확인 및 전처리

In [37]:

#"Departments" 끝에 공백 제거
df.rename(columns={"Departments ":"Departments"},inplace=True)


#원-핫 인코딩 (범주형 변수 처리)
df = pd.get_dummies(df, columns=['Departments','salary'], drop_first=True)

#독립변수(X)와 종속변수(y) 설정
X=df.drop("left",axis=1);
y=df["left"];

#데이터 분할(Train: 80%, Test: 20%)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42);

#데이터 스케일링
scaler=StandardScaler();
X_train_scaled=scaler.fit_transform(X_train);
X_test_scaled=scaler.transform(X_test);

#모델 학습-랜덤 포레스트
model=RandomForestClassifier(n_estimators=100,random_state=42);
model.fit(X_train_scaled,y_train);

In [38]:
#예측
y_pred=model.predict(X_test_scaled);

#평가
accuracy=accuracy_score(y_test,y_pred);
print(f"Accuracy: {accuracy:.2f}");
print("\nConfusion Matrix:");
print(confusion_matrix(y_test,y_pred));
print("\nClassification Report:");
print(classification_report(y_test,y_pred));

# Feature Importance (중요 변수 확인)
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)

Accuracy: 0.99

Confusion Matrix:
[[2286    8]
 [  27  679]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2294
           1       0.99      0.96      0.97       706

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000


Feature Importances:
                    Feature  Importance
0        satisfaction_level        0.33
2            number_project        0.19
4        time_spend_company        0.18
3      average_montly_hours        0.14
1           last_evaluation        0.12
5             Work_accident        0.01
16               salary_low        0.01
15    Departments_technical        0.00
17            salary_medium        0.00
13        Departments_sales        0.00
14      Departments_support        0.00
7         Departments_RandD        0.00
8    Departments_accounting        0.00
6     